In [2]:
# geras
import os
from bs4 import BeautifulSoup
import csv
import re

# Nurodyti aplanką su knygų HTML failais
folder_path = "Grozines_html"

# Sukurti CSV failą ir įrašome antraštes
with open("Grozines_knygu_duomenys.csv", "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = [
        "Pavadinimas",
        "Autorius",
        "Žanras",
        "Metai",
        "Puslapiai",
        "Kaina",
        "Akcijinė kaina",
        "Perskaitytos kaina",
        "ISBN",
        "Leidėjas",
        "Viršelio tipas",
        "Ilgis",
        "Plotis",
        "Storis",
        "Kalba",
        "Vertėjas",
        "Įvertinimų skaičius",
        "Žvaigždučių skaičius",
        "Aprašymas"
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Ciklu einati per kiekvieną failą aplanke
    for file_name in os.listdir(folder_path):
        if "knygu-rinkinys" in file_name.lower():
            print(f"Failas '{file_name}' yra rinkinys ir nebus įtrauktas.")
            continue

        file_path = os.path.join(folder_path, file_name)
        # Atidaryti ir nuskaityti HTML failo turinį
        with open(file_path, "r", encoding="utf-8") as file:
            html_content = file.read()
        # Naudoti BeautifulSoup HTML analizavimui
        soup = BeautifulSoup(html_content, "html.parser")
        book_soup = soup.find(class_="col-12 about-product-wrapper")
        if book_soup:
            book_soup_1 = book_soup.find("div", class_="col-12 col-xl-4 product-img-wrapper product-photo -order-1 order-xl-0")
            book_soup_2 = book_soup.find("div", class_="col-12 col-xl-8 book-descript-wrapper pl-xl-5 -order-0 order-xl-1")
            book_soup_3 = book_soup.find("div", class_="checkout-block")

            if book_soup_1 and book_soup_2 and book_soup_3:
                # Ištraukti informaciją apie knygą
                # Pavadinimas
                title_tag = book_soup.find("span", class_="book-title-raw")
                if title_tag:
                    title = title_tag.text.strip()
                    # print(title)
                else:
                    None
                # Autorius
                author_tag = book_soup.find("a", class_="book-author")
                if author_tag:
                    author = author_tag.text.strip()
                    # print(author)
                else:
                    None
                # Žanras
                genre_tag = soup.find("li", class_="breadcrumb-item d-none d-lg-flex").find("span", itemprop="name")
                if genre_tag:
                    genre = genre_tag.text
                    # print(genre)
                else:
                    None
                # Metai
                for li in book_soup_1.find_all("li"):
                    if "Metai" in li.text:
                        year = li.text.replace("Metai:", "").strip()
                # print(year)
                # Puslapiai
                for li in book_soup_1.find_all("li"):
                    if "Puslapiai" in li.text:
                        pages = li.text.replace("Puslapiai:", "").strip()
                # print(pages)
                # Kaina
                price_tag = book_soup_3.find("div", class_="product-price-wrapper product-price-xxl-horizontal mb-3")
                if price_tag:
                    price = price_tag.text.replace(",", ".").replace("€", "").split()[-1]
                else:
                    price = 0
                # print(price)
                # Akcijinė kaina
                promotional_price_tag = book_soup_3.find(
                    "div", class_="product-price-wrapper product-price-xxl-horizontal mb-3"
                )
                if promotional_price_tag:
                    promotional_price = promotional_price_tag.text.replace(",", ".").replace("€", "").split()[0]
                else:
                    promotional_price = 0
                # print(promotional_price)
                # Perskaitytos kaina
                readed_price_tag = book_soup_2.find("span", class_="book-price")
                if readed_price_tag:
                    readed_price = readed_price_tag.text.replace(",", ".").replace("€", "").strip()
                else:
                    readed_price = 0
                # print(readed_price)
                # ISBN
                isbn = None
                for li in book_soup_1.find_all('li'):
                    if "ISBN" in li.text:
                        isbn = li.text.replace("ISBN:", "").strip()
                # Leidėjas
                for li in book_soup_1.find_all("li"):
                    if "Leidėjas:" in li.text:
                        publisher = li.text.replace("Leidėjas:", "").strip()
                        # print(publisher)
                # Kalba
                for li in book_soup_1.find_all("li"):
                    if "Kalba:" in li.text:
                        language = li.text.replace("Kalba:", "").strip()
                        # print(language)
                # Vertėjas
                for li in book_soup_1.find_all("li"):
                    translator = None
                    if "Vertėjas:" in li.text:
                        translator = li.text.replace("Vertėjas:", "").strip()
                        # print(translator)

                # Knygos viršelio formatas
                for li in book_soup_1.find_all("li"):
                    if "Formatas:" in li.text:
                        format = li.text.replace("Formatas:", "").strip()
                        match = re.search(r'(\d+)\s*x\s*(\d+)\s*x?\s*(\d*,?\d*)\s*cm.*(kieti|minkšti) viršeliai', format)
                        if match:
                            ilgis = float(match.group(1))
                            plotis = float(match.group(2))
                            storis = match.group(3)
                            virselis = match.group(4)                            
                            storis = float(storis.replace(',', '.')) if storis else None
                            # print(f"Ilgis: {ilgis}, Plotis: {plotis}, Storis: {storis}, Viršelio tipas: {virselis}")
                        else:
                            print("format not found.")
                # Įvertinimų skaičius
                review_tag = soup.find("div", class_="col-auto item-rating-user")
                if review_tag:
                    review = review_tag.text.split()[0]
                else:
                    review = 0
                stars_tag = book_soup_2.find("span", class_="book-properties book-stars")
                if stars_tag:
                    filled_stars = len(stars_tag.find_all("i", class_="ico-star-fill"))
                    empty_stars = len(stars_tag.find_all("i", class_="ico-star-fill empty"))
                    stars = filled_stars - empty_stars
                else:
                    None
                # print('žvaigždutės',stars)
                # Prieinamumas
                # status_tag1_list = []
                # for strong in book_soup_3.find_all('strong'):
                #     status_tag1_list.append(book_soup_3.find('strong').text)
                # print(status_tag1_list[-1])
                # if  "Išparduota" in status_tag1.text or "Perskaityta knyga:" in status_tag1.text:
                #     status = status_tag1.text.strip().replace(':', '')
                # status_tag2 = book_soup_3.find('span', class_ = 'in-store-statuses-text')
                # if status_tag2:
                #     status = status_tag2.text
                #     print(status)
                # else:
                #     None
                # print(status)
                # status = book_soup.find('span', class_="in-store-statuses-text").text.strip() if book_soup.find('span', class_="in-store-statuses-text") else None

                # if status == 'Išparduota':
                #     readed = 'Nėra'
                # Perskaityta knyga
                # if book_soup.find('div', class_ = 'note note-warning mb-3 p-3'):
                #     readed = book_soup.find('div', class_ = 'note note-warning mb-3 p-3').text.split()[0]
                #     print(readed)
                # else: "Nėra"
                # Aprašymas
                description_tag = book_soup_2.find("section", class_="product-descript")
                if description_tag:
                    description = description_tag.text.replace("Aprašymas", "").strip()
                # print(description)
                # print('*******')
                # Įrašyti informaciją į CSV failą
                writer.writerow(
                    {
                        "Pavadinimas": title,
                        "Autorius": author,
                        "Žanras": genre,
                        "Kaina": price,
                        "Akcijinė kaina": promotional_price,
                        "Perskaitytos kaina": readed_price,
                        "Metai": year,
                        "Puslapiai": pages,
                        "ISBN": isbn,
                        "Leidėjas": publisher,
                        "Viršelio tipas": virselis,
                        "Ilgis": ilgis,
                        "Plotis": plotis,
                        "Storis": storis,
                        "Kalba": language,
                        "Vertėjas": translator,
                        # "Prieinamumas": status,
                        "Įvertinimų skaičius": review,
                        "Žvaigždučių skaičius": stars,
                        "Aprašymas": description
                    }
                )
            else:
                print("One or more parts are missing.")
        else:
            print("Main container 'about-product-wrapper' not found.")
print("Informacija sėkmingai išsaugota į 'Grozines_data.csv'")

format not found.
format not found.
format not found.
format not found.
format not found.
Failas '2-knygu-rinkinys--liucernos-liutai---snipu-vadas-8i4dz.html' yra rinkinys ir nebus įtrauktas.
format not found.
format not found.
Failas '3-knygu-rinkinys--liucernos-liutai---snipu-vadas-----kupranugario--klubas-1lu2d.html' yra rinkinys ir nebus įtrauktas.
format not found.
format not found.
format not found.
format not found.
Failas '7-knygu-rinkinys-raganius--magiskame-pasaulyje-kupiname-pabaisu--ant-jo-peciu---atsakomybe-uz-nepaprasta-viso-pasaulio-likima-1jv4k.html' yra rinkinys ir nebus įtrauktas.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found.
format not found